In [10]:
import csv
import pickle
import datetime
from sklearn.model_selection import train_test_split

In [11]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png','retina'}

In [12]:
def import_data():
    df = pd.read_csv('data/final_df.csv')
    df = df.iloc[:,1:]
    
    
    return df

In [ ]:
# data = pd.read_csv('data/data.csv')
# data=data.iloc[:,1:]

In [ ]:
# new_order = ['date', 'store_nbr', 'item_nbr', 'station_nbr', 'tmax', 'tmin',
#        'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
#        'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
#        'avgspeed', 'resultspeed', 'resultdir', 'TS', 'GR', 'RA', 'DZ', 'SN',
#        'SG', 'GS', 'PL', 'FG+', 'FG', 'BR', 'UP', 'HZ', 'FU', 'DU', 'SQ', 'FZ',
#        'MI', 'PR', 'BC', 'BL', 'VC', 'units']

In [ ]:
# data = data[new_order]

In [ ]:
# data.columns = ['date', 'store_nbr', 'item_nbr', 'station_nbr', 'tmax', 'tmin', 'tavg',
#        'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise', 'sunset',
#        'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'avgspeed',
#        'resultspeed', 'resultdir', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS',
#        'PL', 'FG2', 'FG', 'BR', 'UP', 'HZ', 'FU', 'DU', 'SQ', 'FZ', 'MI', 'PR',
#        'BC', 'BL', 'VC', 'units']

In [13]:
def fulldf(df):
    
    df.snowfall = df.snowfall.replace(["T","  T"],0.05)
    df.preciptotal = df.preciptotal.replace(["T","  T"],0.005)
    df.depart = pd.to_numeric(df.depart, errors='coerce')
    
    df = type_change_numeric(df,[ 'store_nbr', 'item_nbr', 'units', 'station_nbr', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'avgspeed', 'resultspeed', 'resultdir' ])
    df['date'] = pd.to_datetime(df['date'])
                                   
    
    df["day_of_year"] = df['date'].dt.dayofyear
    df["year"] = df['date'].dt.year
    df["month"] = df["date"].dt.month
    
    
    for idx in range(5, 8):
        df.iloc[:,idx].fillna(df.groupby(["day_of_year","store_nbr"])[df.columns[idx]].\
                              transform('mean'), inplace=True)

    for idx in range(16, 23):
        df.iloc[:,idx].fillna(df.groupby(["day_of_year","store_nbr"])[df.columns[idx]].\
                              transform('mean'), inplace=True)
    add_depart1(df)
    
    return df

# for column in item37.columns:
#     item37[column].interpolate()

In [14]:
def type_change_numeric(df, ls = []): 
    #ls에 있는 column name은 numeric형으로 바꾸지 않는다.
    cols = df.columns
    for i in cols:
        if i in ls:
            #df = df.replace(["M",None], '')
            df.snowfall = df.snowfall.replace(["T","  T"],0.05)
            df.preciptotal = df.preciptotal.replace(["T","  T"],0.005)
            df[i] = pd.to_numeric(df[i], errors='coerce')
        
    return df

In [15]:
def add_depart1(x):
    x.depart.fillna(x.tavg - x.groupby(["day_of_year","store_nbr"])["tavg"].transform('mean'),inplace = True)
    x.depart = x.depart.round(2)
    return x.sort_values(["store_nbr","date","item_nbr"])

In [16]:
def reorder_df(df):
    #Column 정렬 (y값을 마지막으로 ) 후 FG+ -> FG2 변환 (formula에 인식시키기위해 )
    new_order =  ['date', 'store_nbr', 'item_nbr', 'station_nbr', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'avgspeed', 'resultspeed', 'resultdir', 'TS', 'GR', 'RA', 'DZ', 'SN',
       'SG', 'GS', 'PL', 'FG+', 'FG', 'BR', 'UP', 'HZ', 'FU', 'DU', 'SQ', 'FZ',
       'MI', 'PR', 'BC', 'BL', 'VC', 'day_of_year', 'year', 'month', 'units' ]
    df = df[new_order]
    df.columns = ['date', 'store_nbr', 'item_nbr', 'station_nbr', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'avgspeed', 'resultspeed', 'resultdir', 'TS', 'GR', 'RA', 'DZ', 'SN',
       'SG', 'GS', 'PL', 'FG2', 'FG', 'BR', 'UP', 'HZ', 'FU', 'DU', 'SQ', 'FZ',
       'MI', 'PR', 'BC', 'BL', 'VC', 'day_of_year', 'year', 'month', 'units' ]
    
    return df

In [17]:
def add_cat_columns(df):
    df['date'] = pd.to_datetime(df['date'])
#     df['sunrise'] = pd.to_datetime(df['sunrise'], format='%H%M')
#     df['sunset'] = pd.to_datetime(df['sunset'], format='%H%M')

    blackfriday = ["2012-11-21","2012-11-22","2012-11-23", "2012-11-24","2012-11-25",
              "2013-11-27","2013-11-28", "2013-11-29","2013-11-30","2013-11-31",
              "2014-11-26", "2014-11-27", "2014-11-28","2014-11-29","2014-11-30"]
    df["week_day_name"] = df['date'].dt.weekday_name
    df['weekend'] = False
    df.weekend[df['week_day_name'] == 'Sunday'] = True
    df.weekend[df['week_day_name'] == 'Saturday'] = True
    df.weekend[df['week_day_name'] == 'Friday'] = True
    df["is_blackfriday"] = df.date.apply(lambda x : str(x)[:10] in blackfriday).astype(int)
    
    df['hardrain'] = [(((4 if i > 8 else 3) if i > 6 else 2) if i > 1 else 1) if i > 0 else 0 for i in df['preciptotal']]
    # rain 1 snow 2
    # 득정조건열 추가 ()
    df['hardsnow'] = [(( 3 if i > 3.5 else 2) if i > 1 else 1) if i > 0 else 0 for i in df['snowfall']]
    
    df['log_units'] = df.units.apply(lambda x: np.log(x + 1)).astype(float)
    return df

In [18]:
def df_sampling(df):
    new_order = ['date', 'store_nbr', 'item_nbr', 'station_nbr', 'tmax', 'tmin', 'tavg',
       'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'preciptotal', 'stnpressure', 'sealevel', 'avgspeed',
       'resultspeed', 'resultdir', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS',
       'PL', 'FG2', 'FG', 'BR', 'UP', 'HZ', 'FU', 'DU', 'SQ', 'FZ', 'MI', 'PR',
       'BC', 'BL', 'VC', 'day_of_year', 'year', 'month', 
       'week_day_name', 'weekend', 'is_blackfriday', 'hardrain', 'hardsnow',
       'log_units', 'units' ]
    
    df = df[new_order]
    
    X, y = df.iloc[:,:-1], df.units
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 2018)
    
    train = pd.concat([X_train, y_train] ,axis = 1)
    train = train.sort_values(by=['date', 'store_nbr', 'item_nbr']).reset_index(drop= True)
    y_train = y_train.sort_index()
    y_test = y_test.sort_index()
    
    return train, y_train, y_test
   

In [19]:
data = import_data()
data_t = fulldf(data)
data_t = reorder_df(data_t)
data_t  = add_cat_columns(data_t)

KeyError: "['sunrise' 'sunset' 'resultdir'] not in index"

In [ ]:
del data_t['sunrise']
del data_t['sunset']
del data_t['snowfall']
data_t = data_t.dropna()

In [ ]:
train0, y_train, y_test = df_sampling(data_t)
train = sm.add_constant(train0)

In [ ]:

# X, y = data_t.iloc[:,:-1], data_t.units
# X_train, X_testm, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 2018)
# train = pd.concat([X_train, y_train] ,axis = 1)
# train = train.sort_values(by=['date', 'store_nbr', 'item_nbr']).reset_index(drop= True)

In [ ]:
# X_train['date'] = pd.to_datetime(X_train['date'])
# X_train['sunrise'] = pd.to_datetime(X_train['sunrise'], format='%H%M')
# X_train['sunset'] = pd.to_datetime(X_train['sunset'], format='%H%M')
# X_testm['date'] = pd.to_datetime(X_testm['date'])
# X_testm['sunrise'] = pd.to_datetime(X_testm['sunrise'], format='%H%M')
# X_testm['sunset'] = pd.to_datetime(X_testm['sunset'], format='%H%M')

In [ ]:
# train['log_units'] = train.units.apply(lambda x: np.log(x + 1))

In [ ]:

# train['month'] =train["date"].dt.month
# train['weekend'] = False
# train.weekend[train['week_day_name'] == 'Sunday'] = True
# train.weekend[train['week_day_name'] == 'Saturday'] = True
# train.weekend[train['week_day_name'] == 'Friday'] = True
# train["is_blackfriday"] = train.date.apply(lambda x : str(x)[:10] in blackfriday).astype(int)

In [ ]:
# blackfriday = ["2012-11-21","2012-11-22","2012-11-23", "2012-11-24","2012-11-25",
#               "2013-11-27","2013-11-28", "2013-11-29","2013-11-30","2013-11-31",
#               "2014-11-26", "2014-11-27", "2014-11-28","2014-11-29","2014-11-30"]

In [ ]:
# train[train > 3.5].preciptotal.hist(bins = 20)

In [ ]:
# train['hardrain'] = [(((4 if i > 8 else 3) if i > 6 else 2) if i > 1 else 1) if i > 0 else 0 for i in train['preciptotal']]
# # rain 1 snow 2
# # 득정조건열 추가 ()
# train['hardsnow'] = [(( 3 if i > 3.5 else 2) if i > 1 else 1) if i > 0 else 0 for i in train['snowfall']]

In [ ]:
# del train['sunrise']
# del train['sunset']
# del train['snowfall']
# train = train.dropna()

In [ ]:
train.dtypes

In [ ]:

model_OLS = sm.OLS.from_formula("log_units ~  C(store_nbr) + C(item_nbr) + scale(depart) + scale(dewpoint) + scale(preciptotal) + C(weekend) + C(is_blackfriday) + 0" , data = train)
results_OLS = model_OLS.fit()
print(results_OLS.summary())

In [ ]:
sns.jointplot(results_OLS.fittedvalues, y)
plt.show()

In [ ]:
results_OLS.predict(train)

In [ ]:
resi = (y_test) - np.exp(results_OLS.predict(train))

In [ ]:
model_GLSAR = sm.GLSAR.from_formula(" log_units ~  C(store_nbr) + C(item_nbr)  + scale(dewpoint) + scale(preciptotal) + C(weekend) + C(is_blackfriday) + 0" , data = train, rho=1)
results_GLSAR = model_GLSAR.fit()
print(results_GLSAR.summary())

In [ ]:
plt.figure(figsize= (12, 8))
plt.scatter(   ,results_GLSAR.resid)


In [ ]:
ax1 = plt.subplot(211); sm.graphics.tsa.plot_acf(results_GLSAR.resid, ax=ax1)
plt.xlim(0,5)
ax2 = plt.subplot(212); sm.graphics.tsa.plot_pacf(results_GLSAR.resid, ax=ax2)
plt.xlim(0,5)
plt.tight_layout()
plt.show() 

In [ ]:
model_GLSAR = sm.GLSAR.from_formula("I(np.log((units +1))) ~  C(store_nbr) + C(item_nbr) + scale(depart) + scale(dewpoint) + scale(preciptotal) + C(SN) + C(FG2) + C(weekend) + C(month) + C(year) + C(is_blackfriday) + C(hardsnow) + C(hardrain)" , data = cont_df, rho=1)
results_GLSAR = model_GLSAR.fit()
print(results_GLSAR.summary())